This notebook explores how FunctionBase1D exposes a callable (lambda_fun) for numerical evaluation. It contrasts passing coefficients explicitly with binding coefficients to the expression, and it repeats both cases with uncertainty-bearing parameters. The notebook exists to demonstrate the evaluation path used by regression and uncertainty propagation. After working through it, the reader should understand how lambda_fun expects inputs and how uncertainty values flow through the calculation.


Import the symbolic helper and Jupyter display utilities.


In [1]:
from unc_tools import FunctionBase1D
from IPython.display import display, Latex


Define optional helpers for LaTeX display (useful when inspecting expressions).


In [2]:
def ldisplay(expr):
    # Render a LaTeX expression inside Jupyter.
    display(Latex("$" + expr + "$"))

def print_all_info(expr, ys = [0,"t",5]):
    # Helper for optional inspection of expression metadata and solutions.
    print(f"""
Expresssion :           {expr.expr_str}
Sympy expression:       {expr.expr_sym}
All solutions expr = 0: {expr.sols}
LaTeX expression:       {expr.to_latex_expr()}
""")
    ldisplay(expr.to_latex_expr())

    for y in ys:
        print("-"*20)
        print(f"""
Solutions expr = {y}: {expr.find_sols(y)}
LaTeX solutions for expr = {y}:
{expr.to_latex_sols()}
LaTeX solutions for expr = {y} with uncertainty:
{expr.to_latex_sols(show_unc=True)}
""")
        ldisplay(expr.to_latex_sols(y))
        ldisplay(expr.to_latex_sols(y,show_unc=True))


Evaluate a polynomial by passing coefficients directly to lambda_fun.


In [3]:
from unc_tools import Poly
import uncertainties as unc
import numpy as np


expr = Poly(1)

# lambda_fun expects x plus coefficients when not bound.
func = expr.lambda_fun

x = np.linspace(0,10,20)
y = func(x, 5, 3)

y


array([ 3.        ,  5.63157895,  8.26315789, 10.89473684, 13.52631579,
       16.15789474, 18.78947368, 21.42105263, 24.05263158, 26.68421053,
       29.31578947, 31.94736842, 34.57894737, 37.21052632, 39.84210526,
       42.47368421, 45.10526316, 47.73684211, 50.36842105, 53.        ])

Evaluate a polynomial by passing uncertain coefficients to lambda_fun.


In [4]:
from unc_tools import Poly
import uncertainties as unc
import numpy as np


expr = Poly(1)

# Passing ufloat coefficients propagates uncertainty to y.
func = expr.lambda_fun

x = np.linspace(0,10,20)
y = func(x, unc.ufloat(5,0.5), unc.ufloat(3,0.3))

y


array([3.0+/-0.3, 5.631578947368421+/-0.39906400183720764,
       8.263157894736842+/-0.6058121080411876,
       10.894736842105264+/-0.8445523654936631,
       13.526315789473683+/-1.0945470483251187,
       16.157894736842103+/-1.349556200777934,
       18.789473684210527+/-1.607194696433437,
       21.421052631578945+/-1.866373971248532,
       24.052631578947366+/-2.1265307343156175,
       26.684210526315788+/-2.387345446840167,
       29.31578947368421+/-2.6486237475777274,
       31.94736842105263+/-2.910240777846663,
       34.578947368421055+/-3.172112729550302,
       37.21052631578947+/-3.434181286425229,
       39.84210526315789+/-3.6964046318302417,
       42.473684210526315+/-3.9587520068227997,
       45.10526315789473+/-4.221200286169288,
       47.73684210526316+/-4.483731751065452,
       50.368421052631575+/-4.746332597932208, 53.0+/-5.008991914547277],
      dtype=object)

Bind fixed coefficients to the expression, then evaluate with only x.


In [5]:
from unc_tools import Poly
import uncertainties as unc
import numpy as np


expr = Poly(1)
    
coefs = [5,3]
# Bind coefficients so lambda_fun only expects x.
expr.add_coefs(coefs)

func = expr.lambda_fun

x = np.linspace(0,10,20)
y = func(x)

y


array([ 3.        ,  5.63157895,  8.26315789, 10.89473684, 13.52631579,
       16.15789474, 18.78947368, 21.42105263, 24.05263158, 26.68421053,
       29.31578947, 31.94736842, 34.57894737, 37.21052632, 39.84210526,
       42.47368421, 45.10526316, 47.73684211, 50.36842105, 53.        ])

Bind uncertain coefficients to the expression, then evaluate with only x.


In [6]:
from unc_tools import Poly
import uncertainties as unc
import numpy as np


expr = Poly(1)

coefs = [unc.ufloat(5,0.5), unc.ufloat(3,0.3)]
# Bind uncertain coefficients to propagate errors automatically.
expr.add_coefs(coefs)

func = expr.lambda_fun

x = np.linspace(0,10,20)
y = func(x)

y


array([3.0+/-0.3, 5.631578947368421+/-0.39906400183720764,
       8.263157894736842+/-0.6058121080411876,
       10.894736842105264+/-0.8445523654936631,
       13.526315789473683+/-1.0945470483251187,
       16.157894736842103+/-1.349556200777934,
       18.789473684210527+/-1.607194696433437,
       21.421052631578945+/-1.866373971248532,
       24.052631578947366+/-2.1265307343156175,
       26.684210526315788+/-2.387345446840167,
       29.31578947368421+/-2.6486237475777274,
       31.94736842105263+/-2.910240777846663,
       34.578947368421055+/-3.172112729550302,
       37.21052631578947+/-3.434181286425229,
       39.84210526315789+/-3.6964046318302417,
       42.473684210526315+/-3.9587520068227997,
       45.10526315789473+/-4.221200286169288,
       47.73684210526316+/-4.483731751065452,
       50.368421052631575+/-4.746332597932208, 53.0+/-5.008991914547277],
      dtype=object)

Evaluate a nonlinear expression with uncertainty-bearing coefficients.


In [7]:
expr = FunctionBase1D("exp(x)*a + b")

coefs = [unc.ufloat(10,0.1), unc.ufloat(5,0.5)]
# Nonlinear expression with uncertain parameters.
expr.add_coefs(coefs)

func = expr.lambda_fun

x = np.linspace(0,1,20)
y = func(x)

y


array([15.0+/-0.5099019513592785, 15.540412425918024+/-0.5109892654555913,
       16.11002941084471+/-0.5121945678655558,
       16.710429205438224+/-0.5135303449822202,
       17.343275350983443+/-0.5150103344972987,
       18.01032128860335+/-0.5166496356364423,
       18.713415217558058+/-0.5184648258974907,
       19.45450521609223+/-0.5204740839890145,
       20.23564463901954+/-0.5226973184900114,
       21.05899780699929+/-0.5251563015489016,
       21.92684600326856+/-0.5278748067125736,
       22.841593794445277+/-0.530878749727844,
       23.80577569291529+/-0.5341963308945733,
       24.822063179263147+/-0.5378581772812141,
       25.89327210420374+/-0.5418974828526424,
       27.022370490523553+/-0.5463501443233892,
       28.212486756648488+/-0.5512548903572269,
       29.46691838462353+/-0.556653401609999,
       30.7891410565208+/-0.5625904190824019,
       32.182818284590454+/-0.5691138383393137], dtype=object)